# Network introduction
Main parts of the course:
1. networks definitions and network measures
2. networks in time  
3. networks from data

This notebook was inspired from:
* Big data course from 2019 Marc and Liubov https://github.com/Big-data-course-CRI/materials_big_data_cri_2019
* Bruno Gonçalves / Data4Sci: https://github.com/DataForScience/Networks
* Michael Szell data course https://github.com/mszell



# Network libraries

If we want to use the network theory calculation in python, it is easier to load a library.

https://networkx.org/

If you use this library, you do not need to deal with all the classes encoding of network structures yourself.


# Structure of the notebook
1. First we add and encode networks ourselves NOT using precoded class of networks
2. We can use networkx class for network object (see below cells).

In [1]:
from collections import Counter
from pprint import pprint

import numpy as np
import matplotlib.pyplot as plt

# 1. Network definition and generation
We can use edge list for generation of a network.

Let us start by defining a list of edges. This will give us our first "dataset" to work with

In [9]:
edge_list = [
    ('A', 'B'), # one friendship encoded
    ('A', 'C'), # another friendship encoded...
    ('A', 'E'),
    ('B', 'C'),
    ('C', 'D'),
    ('C', 'E'),
    ('D', 'E')]

# please, try to create your own edge_list:
edge_list1 = [(1,2), (1,3), (5,6), (10, 3)]

edge_list2 = [(1,2), (1,3), (2,4)]

number_edges = len(edge_list1)
print('number of edges in the first list is ', number_edges)


print('type of edge list ', type(edge_list))

number of edges in the first list is  4
type of edge list  <class 'list'>


In [ ]:
print('How to see elements in the edge list?')

print(type(edge_list[1]))

print(edge_list[0])

print(edge_list[1])

print(edge_list[1][1])

print(type(edge_list[1][1]))


<class 'tuple'>
('A', 'B')
('A', 'C')
C
<class 'str'>


This is a particularly useful representation as many datasets are distributed in this (or a closely related) format. From this list, we can easily measure the number of edges that constitute our network.

Its main limitations are that it has no way to explictly take into account disconnected nodes (it only accounts for nodes that are part of edges) and no indication on whether it is directed or not.

In [10]:
number_edges = len(edge_list)
print(number_edges)

7


# How to calculate number of nodes in our network?

Mind that some nodes are part of several edges!!!


To get the number of node is a bit trickier. We must go edge by edge and keep track of all new nodes. For efficiency, we use a set to automatically remove duplicates

In [11]:
# we gave the information to computer which contains only information about edges
nodes = set() # in the beginning we have empty set of nodes

for edge in edge_list: # for loop for our network
    print(edge)
    nodes.update(edge) # loop through each link in our list of edges
    # function called 'update' is adding only new elements into the set of nodes
    print(nodes)

number_nodes = len(nodes)
print(number_nodes)

('A', 'B')
{'B', 'A'}
('A', 'C')
{'C', 'B', 'A'}
('A', 'E')
{'E', 'C', 'B', 'A'}
('B', 'C')
{'E', 'C', 'B', 'A'}
('C', 'D')
{'E', 'D', 'C', 'B', 'A'}
('C', 'E')
{'E', 'D', 'C', 'B', 'A'}
('D', 'E')
{'E', 'D', 'C', 'B', 'A'}
5


Now we know that we have 5 nodes and 7 edges in our network. The node ids are:

In [13]:
nodes

print('length of nodes set', len(nodes))

length of nodes set 5


Using Adjacency List for network generation.

**Important:** you may generate and build network from your data using adjacency list.

A closely related data structure to the edge list is the adjacency list. In this formulation, we use a dictionary to map each node to its set of neighbors

In [ ]:
adj_list = {}

for node_i, node_j in edge_list:
    if node_i not in adj_list:
        adj_list[node_i] = set()

    adj_list[node_i].add(node_j)

Our adjaceny list is then:

In [ ]:
pprint(adj_list)

print(type(adj_list))

{'A': {'C', 'B', 'E'}, 'B': {'C'}, 'C': {'E', 'D'}, 'D': {'E'}}
<class 'dict'>


In this approach we inherently assumed that our network is directed (or, equivalently, that both edge directions are present in the data). To generate an undirected version we must make a simple modification to our code to manually add the opposite direction edge

In [ ]:
adj_list = {}

for node_i, node_j in edge_list:
    if node_i not in adj_list:
        adj_list[node_i] = set() # 'set' is used to prevent accidental multiple edges

    adj_list[node_i].add(node_j)

    # Manually add the opposite direction edge
    if node_j not in adj_list:
        adj_list[node_j] = set()

    adj_list[node_j].add(node_i)

The undirected adjacency list represenation is then:

In [ ]:
pprint(adj_list)

{'A': {'C', 'E', 'B'},
 'B': {'C', 'A'},
 'C': {'E', 'A', 'D', 'B'},
 'D': {'C', 'E'},
 'E': {'C', 'A', 'D'}}


# Adjacency Matrix
Generation of a network using matrix.

We now move on to generating an Adjacency Matrix view of the network. For this we must have two things:

- the number of nodes in the network
- A mapping between the original node ids and a sequential numerical ID

We start by building out the numerical ID mapping. As we do, we get the number of nodes for free

In [14]:
node_id = {}
node_count = 0

# let us create adjacency matrix from scratch

for node_i, node_j in edge_list:
    if node_i not in node_id:
        node_id[node_i] = node_count
        node_count += 1

    # Make sure we have an id for both nodes
    # This is necessary, irregardless of whether the network is directed or undirected
    if node_j not in node_id:
        node_id[node_j] = node_count
        node_count += 1

We can check that each of the original node ids is correctly mapped to a sequential number

In [ ]:
node_id

{'A': 0, 'B': 1, 'C': 2, 'D': 4, 'E': 3}

Finally, we are ready to build our adjacency matrix. We start by declaring the data structurewe will use

In [ ]:
adj_matrix = np.zeros((node_count, node_count), dtype='int')

And we can now populate the matrix entries. For generality, we'll include a flag to control wether or not the graph is directed. As we don't have any weights associated with our edges, we assign to each of them a value of 1.

In [ ]:
is_directed = False

for node_i, node_j in edge_list:
    # Get the correct node ids
    node_i = node_id[node_i]
    node_j = node_id[node_j]

    adj_matrix[node_i, node_j] = 1 # Unweighted network

    if not is_directed:
        adj_matrix[node_j, node_i] = 1 # Undirected networks

Our Adjacency Matrix is then:

In [ ]:
adj_matrix

array([[0, 1, 1, 1, 0],
       [1, 0, 1, 0, 0],
       [1, 1, 0, 1, 1],
       [1, 0, 1, 0, 1],
       [0, 0, 1, 1, 0]])

As we can see, the Adjacency matrix representation is very wasteful, using 25 values to store a 7 (14) edge network plus 5 dictionary entries for the id mappings.

# Adjacency Dictionary

The final graph representation we will explore is the Adjacency Dict. This is a generalization of the Adjacency List covered above that is a bit more flexible and is able to accuratly account for diconnected nodes, weights, etc. For this we will need two datastructures, one to store node information and one for edges. For the sake of flexbility, we will use dicts for both.

In [ ]:
nodes = {}
edges = {}
is_directed = False

And we can now populate them from our original edge_list

In [ ]:
for node_i, node_j in edge_list:
    if node_i not in nodes:
        nodes[node_i] = {}
        edges[node_i] = {}

    if node_j not in nodes:
        nodes[node_j] = {}

        if not is_directed:
            edges[node_j] = {}

    edges[node_i][node_j] = {}

    if not is_directed:
        edges[node_j][node_i] = {}

Our set of nodes is:

In [ ]:
nodes

{'A': {}, 'B': {}, 'C': {}, 'D': {}, 'E': {}}

Where we chose to use dictinaries to allow for the storage of node attributes. Further, our edges are now:

In [ ]:
edges

{'A': {'B': {}, 'C': {}, 'E': {}},
 'B': {'A': {}, 'C': {}},
 'C': {'A': {}, 'B': {}, 'D': {}, 'E': {}},
 'D': {'C': {}, 'E': {}},
 'E': {'A': {}, 'C': {}, 'D': {}}}

Where we once again opted to associate a dictionary to each edge. This gives us the ability to associate edge information (such as weights, etc) to each node.

As we can see, this is the most flexible representation and can be easily converted to one of the other representations if necessary so we will use this approach for the rest of the lecture.

Let us try to use networkx library in the next notebook.